In [1]:
"""
Based on the tflearn CIFAR-10 example at:
https://github.com/tflearn/tflearn/blob/master/examples/images/convnet_cifar10.py
"""

from __future__ import division, print_function, absolute_import

from skimage import color, io
from scipy.misc import imresize
import numpy as np
from sklearn.cross_validation import train_test_split
import os
from glob import glob

import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
from tflearn.metrics import Accuracy

import csv
import cv2
import pandas as pd
from tqdm import tqdm



/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [2]:
import sys
###################################
### Import picture files 
###################################
filenames = os.listdir(os.getcwd()+'/data/X_Train')
filenames = filter(lambda x: x.endswith('.jpg'), filenames)
filenames = map(lambda x: int(x.split('.')[0]), filenames)
filenames.sort()
filenames = map(lambda x: str(x)+".jpg", filenames)

n_files = len(filenames)
print(n_files)

size_image = 64

allX = np.zeros((n_files, size_image, size_image, 3), dtype='float64')
ally = np.zeros(n_files)
count = 0


lable_csv = pd.read_csv("data/Y_Train.csv")
     
for filename in tqdm(filenames):
    if filename.endswith('.jpg'):
#         sys.stdout.write('Loading %s\r' % (filename))
#         sys.stdout.flush()
        img = cv2.imread(os.getcwd()+'/data/X_Train/'+filename)
        img = imresize(img, (size_image, size_image, 3))
        ally[count] = int(lable_csv[lable_csv["Image"]==filename]["Label"])
        allX[count] = np.array(img)
        count += 1

  0%|          | 0/5907 [00:00<?, ?it/s]

5907


100%|██████████| 5907/5907 [00:40<00:00, 145.67it/s]


In [3]:

###################################
# Prepare train & test samples
###################################

# test-train split   
X, X_test, Y, Y_test = train_test_split(allX, ally, test_size=0.1, random_state=42)

# encode the Ys
Y = to_categorical(Y, 2)
Y_test = to_categorical(Y_test, 2)


###################################
# Image transformations
###################################

# normalisation of images
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Create extra synthetic training data by flipping & rotating images
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

###################################
# Define network architecture
###################################

# Input is a 32x32 image with 3 color channels (red, green and blue)
network = input_data(shape=[None, size_image, size_image, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)

# 1: Convolution layer with 32 filters, each 3x3x3
conv_1 = conv_2d(network, size_image/2, 3, activation='relu', name='conv_1')

# 2: Max pooling layer
network = max_pool_2d(conv_1, 2)

# 3: Convolution layer with 64 filters
conv_2 = conv_2d(network, size_image, 3, activation='relu', name='conv_2')

# 4: Convolution layer with 64 filters
conv_3 = conv_2d(conv_2, size_image, 3, activation='relu', name='conv_3')

# 5: Max pooling layer
network = max_pool_2d(conv_3, 2)

# 6: Fully-connected 512 node layer
network = fully_connected(network, size_image*8, activation='relu')

# 7: Dropout layer to combat overfitting
network = dropout(network, 0.5)

# 8: Fully-connected layer with two outputs
network = fully_connected(network, 2, activation='softmax')

# Configure how the network will be trained
acc = Accuracy(name="Accuracy")
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001, metric=acc)

# Wrap the network in a model object
model = tflearn.DNN(network, checkpoint_path='model_cat_dog_11.tflearn', max_checkpoints = 3,
                    tensorboard_verbose = 3, tensorboard_dir='tmp/tflearn_logs11/')

###################################
# Train model for 100 epochs
###################################
model.fit(X, Y, validation_set=(X_test, Y_test), batch_size=500,
      n_epoch=10, run_id='model_cat_dog_11', show_metric=True)

model.save('model_cat_dog_11_final.tflearn')

Training Step: 109  | total loss: 0.55555 | time: 54.968s
| Adam | epoch: 010 | loss: 0.55555 - Accuracy: 0.7046 -- iter: 5000/5316
Training Step: 110  | total loss: 0.55640 | time: 62.841s
| Adam | epoch: 010 | loss: 0.55640 - Accuracy: 0.7030 | val_loss: 0.54192 - val_acc: 0.7208 -- iter: 5316/5316
--


In [4]:

filenames_test = os.listdir(os.getcwd()+'/data/X_Test')
filenames_test = filter(lambda x: x.endswith('.jpg'), filenames_test)
filenames_test = map(lambda x: int(x.split('.')[0]), filenames_test)
filenames_test.sort()
filenames_test = map(lambda x: str(x)+".jpg", filenames_test)

resultX = np.zeros((len(filenames_test), size_image, size_image, 3), dtype='float64')
resultCount = 0

for filename in filenames_test:
    if filename.endswith('.jpg'):
#         print(filename)
        img = cv2.imread("data/X_Test/"+filename)
        img = imresize(img, (size_image, size_image, 3))
        resultX[resultCount] = np.array(img)
        resultCount += 1
result =  model.predict(resultX)
# print(resultX)
#         writer.writerow( (filename, result) )




In [5]:
f = open("result11.csv", 'wt')
writer = csv.writer(f)
writer.writerow( ('Image', 'Label') )

for i in xrange(len(filenames_test)):
    writer.writerow((filenames_test[i], np.argmax(result[i])))


f.close()